# AGH Modelling of Physical Systems Lab 6 (2025 April 8th)
Sylwester Arabas (sylwester.arabas@agh.edu.pl) & Emma Ware (ecware@ucdavis.edu) 

**1. particle-resolved random-sampled state representation for an advection problem**

In [ ]:
""" notebook code targetting 100% `nbqa pylint` score! :) """

from types import SimpleNamespace
import numpy as np
from matplotlib import pyplot
import scipy
from open_atmos_jupyter_utils import show_plot

In [ ]:
PARAMS_P = SimpleNamespace(
    n_part=10000,
    norm=1e10,
    dist=scipy.stats.norm(loc=250, scale=50),
)

PARAMS_X = SimpleNamespace(
    span=1000,
    n_cell=100,
)
PARAMS_X.step = PARAMS_X.span / PARAMS_X.n_cell

RNG = np.random.default_rng(seed=44)

In [ ]:
def sample(*, params_p, params_x, rng):
    """ samples a particle population specified onto a grid and returns
    a collection indexed by sampling type, with values composed of `cell` and `mult`
    integer-valued arrays with cell IDs and multiplicities, respectively """
    u01 = np.full_like(params_p.n_part, np.nan)
    u01 = rng.uniform(0, 1, size = params_p.n_part)
    return {
        k: {
            'cell': (v['cell'] / params_x.step).astype(int),
            'mult': np.round(v['mult'] * params_p.norm).astype(int),
        }
        for k,v in
        {
            'sampling: uniform random in x': {
                #'cell': np.full_like(u01, np.nan),
                'cell' : (u01*params_x.span),
                #'mult': np.full_like(u01, np.nan),
                'mult': (params_p.dist.pdf(u01 * params_x.span)* params_x.span/params_p.n_part),
            },
            'sampling: constant multiplicity': {
                #'cell': np.full_like(u01, np.nan),
                'cell': params_p.dist.ppf(u01),
                #'mult': np.full_like(u01, np.nan),
                'mult': np.ones_like(u01)/params_p.n_part,
            }
        }.items()
    }

PARTICLES = sample(params_p=PARAMS_P, params_x=PARAMS_X, rng=RNG)

In [ ]:
def plot(*, particles, params_p, params_x, rng, shift=0):
    """ plots the particle state as both a histogram as well as population scatter plot
    (with random coordinates shuffled for the purpose of plotting) """
    _, axs = pyplot.subplot_mosaic(
        [['hist'], ['part']],
        figsize=(11, 6),
        sharex=True,
        tight_layout=True,
    )
    u01 = rng.uniform(0, 1, params_p.n_part)
    scale = params_p.norm / params_p.n_part
    for k in particles:
        axs['hist'].hist(
            x=particles[k]['cell'],
            weights=particles[k]['mult'] / params_p.norm,
            bins=params_x.n_cell,
            range=(0, params_x.n_cell),
            label=f'{k}',
            alpha=.666
        )
        axs['part'].scatter(
            particles[k]['cell'] + u01,
            tuple(reversed(u01)),
            s=.25 + 2 * particles[k]['mult'] / scale
        )
    pdf_x = np.linspace(0, params_x.span, 256)
    pdf_y = params_p.dist.pdf(pdf_x - shift)
    axs['hist'].plot(pdf_x / params_x.step, pdf_y * params_x.step, color='black', label='analytic')
    axs['hist'].set_xlim(0, params_x.n_cell)
    axs['hist'].legend()
    axs['hist'].set_ylabel('pdf(x) ⋅ Δx [1]')
    axs['hist'].set_title(f'{params_p.n_part=}   {params_x.n_cell=}')
    axs['part'].set_xticks(np.arange(params_x.n_cell + 1))
    axs['part'].xaxis.set_tick_params(rotation=75)
    axs['part'].set_xlabel('x / Δx [1]')
    axs['part'].set_yticks([])
    axs['part'].set_ylim(0,1)
    for axes in axs.values():
        axes.grid()
    show_plot()

plot(particles=PARTICLES, params_p=PARAMS_P, params_x=PARAMS_X, rng=RNG)

**2. Monte-Carlo representation of advection dynamics**

In [ ]:
PARAMS_T = SimpleNamespace(
    span = 300,
    n_step = 60,
    wind = 1.5
)
PARAMS_T.step = PARAMS_T.span / PARAMS_T.n_step
PARAMS_T.courant_number = PARAMS_T.wind / PARAMS_X.step * PARAMS_T.step

In [ ]:
def advect(*, params_p, params_t, particles, rng):
    """ performs Monte-Carlo advection of the particles """
    probability_of_shift = abs(params_t.courant_number)
    assert probability_of_shift < 1

    for _ in range(params_t.n_step):
        u01 = rng.uniform(0, 1, params_p.n_part)
        for part in particles.values():
            part['cell'] += probability_of_shift > u01
advect(particles=PARTICLES, params_t=PARAMS_T, params_p=PARAMS_P, rng=RNG)

In [ ]:
plot(
    particles=PARTICLES,
    params_x=PARAMS_X,
    params_p=PARAMS_P,
    rng=RNG,
    shift=PARAMS_T.wind * PARAMS_T.span
)

# Assignment06


Oleksii Korduba (korduba@student.agh.edu.pl)


implementation of 2 dimentional solver

__GOAL__

The objective of this task is to extend a one-dimensional Monte Carlo advection solver to two spatial dimensions.The extension involved handling a 2D particle distribution, defining a constant-in-time 2D flow field, and simulating the transport of scalar quantities (represented by particles).

In [ ]:
PARAMS_P = SimpleNamespace(
    n_part=10000,
    norm=1e10,
    dist=scipy.stats.norm(loc=250, scale=50),)
PARAMS_X = SimpleNamespace(
    span_x=1000,
    x_n_cell=100,)
PARAMS_Y = SimpleNamespace(
    span_y=1000,
    y_n_cell=100,)

PARAMS_X.step_x = PARAMS_X.span_x / PARAMS_X.x_n_cell
PARAMS_Y.step_y = PARAMS_Y.span_y / PARAMS_Y.y_n_cell

RNG = np.random.default_rng(seed=44)

def sample_2_d(*, params_p, params_x, params_y, rng):
    """ This function samples a 2D population onto a grid and returns
    a dictionary with solution """

    u01_x = rng.uniform(0, 1, params_p.n_part)
    u01_y = rng.uniform(0, 1, params_p.n_part)
    return {
        k: {
            'cell_x': (v['cell_x'] / params_x.step_x).astype(int),
            'cell_y': (v['cell_y'] / params_y.step_y).astype(int),
            'mult_x': np.round(v['mult_x'] * params_p.norm).astype(int),
            'mult_y': np.round(v['mult_y'] * params_p.norm).astype(int),
        }
        for k, v in {
            'sampling: uniform random in x and y': {
                'cell_x': u01_x*params_x.span_x,
                'cell_y': u01_y*params_y.span_y,
                'mult_x': params_p.dist.pdf(u01_x * params_x.span_x) *
                 params_x.span_x / params_p.n_part,
                'mult_y': params_p.dist.pdf(u01_y * params_y.span_y) * 
                params_y.span_y / params_p.n_part,
            },
            'sampling: constant multiplicity': {
                'cell_x': params_p.dist.ppf(u01_x),
                'cell_y': params_p.dist.ppf(u01_y),
                'mult_x': np.full_like(u01_x, 1) / params_p.n_part,
                'mult_y': np.full_like(u01_y, 1) / params_p.n_part,
            }
        }.items()
    }

PARTICLES_2_D = sample_2_d(params_p=PARAMS_P, params_x=PARAMS_X, params_y=PARAMS_Y, rng=RNG)


In [ ]:
def plot_2_d(*, particles, params_p, params_x, params_y, rng):
    """ Plots the particle state in 2D. """
    _, axs = pyplot.subplot_mosaic(
    [['hist_x'], ['hist_y'], ['part_x'], ['part_y']],
        figsize=(11, 6),
        sharex=True,
        tight_layout=True,
    )
    u01 = rng.uniform(0, 1, params_p.n_part)
    scale = params_p.norm / params_p.n_part
    for k in particles:
        axs['hist_x'].hist(
            x=particles[k]['cell_x'],
            weights=particles[k]['mult_x'] / params_p.norm,
            bins=params_x.x_n_cell,
            range=(0, params_x.x_n_cell),
            label=f'{k}',
            alpha=.666
        )
        axs['hist_y'].hist(
            x=particles[k]['cell_y'],
            weights=particles[k]['mult_y'] / params_p.norm,
            bins=params_y.y_n_cell,
            range=(0, params_y.y_n_cell),
            label=f'{k}',
            alpha=.666
        )

        axs['part_x'].scatter(
            particles[k]['cell_x'] + u01,
            tuple(reversed(u01)),
            s=.25 + 2 * particles[k]['mult_x'] / scale
        )
        axs['part_y'].scatter(
            particles[k]['cell_y'] + u01,
            tuple(reversed(u01)),
            s=.25 + 2 * particles[k]['mult_y'] / scale
        )

    axs['hist_x'].set_xlim(0, params_x.x_n_cell)
    axs['hist_y'].set_xlim(0, params_y.y_n_cell)
    axs['hist_x'].set_ylabel('pdf(x)  Δx [1]')
    axs['hist_y'].set_ylabel('pdf(x)  Δy [1]')
    axs['hist_x'].set_title(f'{params_p.n_part=} {params_x.x_n_cell=}')
    axs['hist_y'].set_title(f'{params_p.n_part=} {params_y.y_n_cell=}')
    axs['part_x'].set_xticks(np.arange(params_x.x_n_cell + 1))
    axs['part_y'].set_yticks(np.arange(params_y.y_n_cell + 1))
    axs['part_x'].set_xlabel('x / Δx [1]')
    axs['part_y'].set_xlabel('y / Δy [1]')
    axs['part_x'].set_ylim(0, 1)
    axs['part_y'].set_ylim(0, 1)
    axs['part_x'].xaxis.set_tick_params(rotation=75)
    axs['part_y'].xaxis.set_tick_params(rotation=75)
    axs['part_x'].set_yticks([])
    axs['part_y'].set_yticks([])
    for axes in axs.values():
        axes.grid()
    show_plot()
plot_2_d(particles=PARTICLES_2_D, params_p=PARAMS_P, params_x=PARAMS_X, params_y=PARAMS_Y, rng=RNG)



In [ ]:
def plot_2_d_grid(
    *,
    particles: dict,
    params_p,
    params_x,
    params_y,
    rng_shift
) -> None:
    """
    Plot the 2D state of particles on a scatter plot, using cell_x and cell_y
    as coordinates. Point size is proportional to the square root of 
    multiplicity in x and y directions."""
    ax = pyplot.subplots(figsize=(8, 6))[1]
    scale = params_p.norm / params_p.n_part

    for label, data in particles.items():
        x = np.array(data['cell_x'], dtype=float)
        y = np.array(data['cell_y'], dtype=float)

        if rng_shift[1] > 0:
            x += rng_shift[0].uniform(0, rng_shift[1], size=len(x))
            y += rng_shift[0].uniform(0, rng_shift[1], size=len(y))

        mult = np.sqrt(data['mult_x'] * data['mult_y'])

        ax.scatter(
            x, y,
            s=2 + 3 * mult / scale,
            alpha=0.6,
            label=label,
            edgecolors='none'
        )

    ax.set_xlim(0, params_x.x_n_cell)
    ax.set_ylim(0, params_y.y_n_cell)
    ax.set_xticks(np.arange(params_x.x_n_cell + 1))
    ax.set_yticks(np.arange(params_y.y_n_cell + 1))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('2D particle state')

    # Rotate x-axis tick labels
    ax.tick_params(axis='x', labelrotation=75)

    ax.grid(True)
    pyplot.tight_layout()
    pyplot.show()

In [ ]:
rng_and_shift = (RNG, 1.0)
plot_2_d_grid(
    particles=PARTICLES_2_D,
    params_p=PARAMS_P,
    params_x=PARAMS_X,
    params_y=PARAMS_Y,
    rng_shift = rng_and_shift,
)

In [ ]:
PARAMS_T = SimpleNamespace(
span_x = 300,
span_y = 300,
n_step = 50,
x_wind = 1.2,
y_wind = 1.2
)

PARAMS_T.step_x = PARAMS_T.span_x / PARAMS_T.n_step
PARAMS_T.step_y = PARAMS_T.span_y / PARAMS_T.n_step
PARAMS_T.courant_number_x = PARAMS_T.x_wind / PARAMS_X.step_x * PARAMS_T.step_x
PARAMS_T.courant_number_y = PARAMS_T.y_wind / PARAMS_Y.step_y * PARAMS_T.step_y

In [ ]:
def advect_2_d(*, params_p, params_t, particles, rng):
    """ Performs Monte-Carlo advection in 2D """
    prob_x = abs(params_t.courant_number_x)
    prob_y = abs(params_t.courant_number_y)
    assert prob_x < 1 and prob_y < 1
    for _ in range(params_t.n_step):
        u01 = rng.uniform(0, 1, params_p.n_part)
        for part in particles.values():
            part['cell_x'] += np.where(prob_x > u01, 1, 0)
            part['cell_y'] += np.where(prob_y > u01, 1, 0)

advect_2_d(particles=PARTICLES_2_D, params_t=PARAMS_T, params_p=PARAMS_P, rng=RNG)


In [ ]:
rng_and_shift = (RNG, 0.2)
plot_2_d_grid(
    particles=PARTICLES_2_D,
    params_p=PARAMS_P,
    params_x=PARAMS_X,
    params_y=PARAMS_Y,
    rng_shift=rng_and_shift,
)

__Conclusion__
The 2D Monte Carlo advection model successfully captures the primary dynamics of constant velocity advection in both directions. The particle distribution translated in space as expected. The flow field was defined with constant velocities in both the x and y directions, ensuring that particles are displaced linearly over time without deformation. The initial condition consisted of a localized distribution of particles, and the analytic solution after n steps is simply the initial distribution shifted by (n × step_size_x, n × step_size_y).
